<a href="https://colab.research.google.com/github/marcinwolter/MachineLearning2020/blob/main/mnist_mlp_minimal_optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
Optuna example that optimizes a neural network classifier configuration for the
MNIST dataset using Keras.
In this example, we optimize the validation accuracy of MNIST classification using
Keras. We optimize the filter and kernel size, kernel stride and layer activation.
"""
! pip install optuna

import warnings

from keras.backend import clear_session
from keras.datasets import mnist
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.optimizers import RMSprop

import optuna

     |████████████████████████████████| 266kB 14.0MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163kB 44.1MB/s 
     |████████████████████████████████| 81kB 8.7MB/s 
     |████████████████████████████████| 81kB 10.4MB/s 
     |████████████████████████████████| 112kB 52.0MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 133kB 51.3MB/s 
  Created wheel for optuna: filename=optuna-2.3.0-cp36-none-any.whl size=359761 sha256=a2c21df7c6b77e411f939aaf588bf85792e91215e539ac131e713f19ca1b6c3a
  Stored in directory: /root/.cache/pip/wheels/fa/91/19/64b0ec6b964f89c0695a9dc6db6f851d0b54c5381a5c9cadfb
Successfully built optuna
  Created wheel for PrettyTable: filename=prettytable-0.7.2-cp36-none-any.whl size=13700 sha256=0cbb70952d070d7e7b7ee67a267373485e21d2d836b2ba46bace4bfc41718c8f
  Stored in directory: /root/.cache/pip/w

In [2]:

N_TRAIN_EXAMPLES = 3000
N_VALID_EXAMPLES = 1000
BATCHSIZE = 128
CLASSES = 10
EPOCHS = 20


In [3]:
def objective(trial):
    # Clear clutter from previous Keras session graphs.
    clear_session()

    (x_train, y_train), (x_valid, y_valid) = mnist.load_data()

    x_train = x_train.reshape(60000, 784)[:N_TRAIN_EXAMPLES].astype("float32") / 255
    x_valid = x_valid.reshape(10000, 784)[:N_VALID_EXAMPLES].astype("float32") / 255
    y_train = y_train[:N_TRAIN_EXAMPLES]
    y_valid = y_valid[:N_VALID_EXAMPLES]
    

    units = int(trial.suggest_discrete_uniform("units1", 32,1024,1))
    dropout = trial.suggest_uniform('dropout1', .00,.50)
    act = trial.suggest_categorical('activation', ['relu','sigmoid','linear'])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    epochs = int(trial.suggest_discrete_uniform('epochs', 10,50,1))

    model = Sequential()
    model.add(Dense(units, activation='relu', input_shape=(784,)))
    model.add(Dropout(dropout))
    model.add(Dense(units, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(units, activation='relu'))
    model.add(Dropout(dropout))

    model.add(Dense(CLASSES, activation="softmax"))

    # We compile our model with a sampled learning rate.
    model.compile(
        loss="sparse_categorical_crossentropy", optimizer=RMSprop(lr=lr), metrics=["accuracy"]
    )

    model.fit(
        x_train,
        y_train,
        validation_data=(x_valid, y_valid),
        shuffle=True,
        batch_size=BATCHSIZE,
        epochs=epochs,
        verbose=False,
    )

    # Evaluate the model accuracy on the validation set.
    score = model.evaluate(x_valid, y_valid, verbose=0)
    return score[1]


# Find the best parameters

In [4]:


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=600)


[I 2020-12-05 21:20:38,304] A new study created in memory with name: no-name-a5fab272-7b7f-4c8a-a87d-f711ff01544d


11493376/11490434 [==============================] - 0s 0us/step


[I 2020-12-05 21:20:50,700] Trial 0 finished with value: 0.9150000214576721 and parameters: {'units1': 960.0, 'dropout1': 0.14810802964670045, 'activation': 'sigmoid', 'lr': 0.00025055236818330055, 'epochs': 46.0}. Best is trial 0 with value: 0.9150000214576721.
[I 2020-12-05 21:20:54,161] Trial 1 finished with value: 0.8730000257492065 and parameters: {'units1': 449.0, 'dropout1': 0.02350613957868164, 'activation': 'linear', 'lr': 1.9864881070528688e-05, 'epochs': 30.0}. Best is trial 0 with value: 0.9150000214576721.
[I 2020-12-05 21:20:56,171] Trial 2 finished with value: 0.8899999856948853 and parameters: {'units1': 90.0, 'dropout1': 0.24222611364128904, 'activation': 'sigmoid', 'lr': 0.00128435226274464, 'epochs': 12.0}. Best is trial 0 with value: 0.9150000214576721.
[I 2020-12-05 21:20:58,628] Trial 3 finished with value: 0.8119999766349792 and parameters: {'units1': 706.0, 'dropout1': 0.25006974001942256, 'activation': 'linear', 'lr': 1.667203494458433e-05, 'epochs': 17.0}. Bes

In [5]:

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

Number of finished trials: 100
Best trial:
  Value: 0.9380000233650208
  Params: 
    units1: 253.0
    dropout1: 0.2670154265928311
    activation: linear
    lr: 0.004066744279868047
    epochs: 32.0


## Visualizing the Optimization History

In [6]:
from optuna.visualization import plot_optimization_history

plot_optimization_history(study)

## Visualizing High-dimensional Parameter Relationships

In [7]:
from optuna.visualization import plot_parallel_coordinate

plot_parallel_coordinate(study)

## Visualizing Parameter Relationships

In [8]:
from optuna.visualization import plot_contour

plot_contour(study)

### Selecting Parameters to Visualize

In [9]:
plot_contour(study, params=['units1', 'dropout1', 'lr','epochs'])

## Visualizing Individual Parameters

In [10]:
from optuna.visualization import plot_slice

plot_slice(study)

### Selecting Parameters to Visualize

In [11]:
plot_slice(study, params=['units1', 'dropout1'])

## Visualizing Parameter Importances

In [12]:
from optuna.visualization import plot_param_importances

plot_param_importances(study)